In [2]:
## Saving all the files in one location.
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path) 

Mounted at /content/mnt


In [ ]:
#!pip install --target=$nb_path fasttext
#!pip install --target=$nb_path pattern

In [ ]:
## We have build a classification model based on the facebook's FastText library.

In [3]:
import fasttext
import pandas as pd
import re

from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [4]:
## Bring the text data into the colab space.
! gdown --id 1bU_LWVC1wRm_eMUXhQpJfQGIFbjX8UuA

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1bU_LWVC1wRm_eMUXhQpJfQGIFbjX8UuA
To: /content/CPM1.txt
100% 16.8k/16.8k [00:00<00:00, 38.0MB/s]


In [5]:
#Split into train and test data set.
txt_file = pd.read_csv('/content/CPM1.txt',error_bad_lines=False,header=None)
train_file = txt_file[txt_file.index <= 0.6*max(txt_file.index)]
test_file = txt_file[txt_file.index > 0.6*max(txt_file.index)]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 94: expected 1 fields, saw 2\nSkipping line 223: expected 1 fields, saw 8\n'


In [6]:
train_file.to_csv('train.txt',index = False,sep =' ',header = None, quoting =None,quotechar = " ",escapechar = " ")
test_file.to_csv('test.txt',index = False,sep =' ',header = None, quoting =None,quotechar = " ",escapechar = " ")

In [7]:
train_txt = pd.read_csv('train.txt',header = None)
test_txt = pd.read_csv('test.txt',header = None)

In [8]:
train_txt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       216 non-null    object
dtypes: object(1)
memory usage: 1.8+ KB


In [9]:
label_ = []
message = []

for i in range(0,len(train_txt)):
  mssg = str(train_txt[train_txt.index==i])
  mssg_ = re.split(r"\s+", mssg,maxsplit=4)
  label_.append(mssg_[3])
  message.append(mssg_[4])

In [10]:
train_df = pd.DataFrame({'Message':message,'Label':label_})

In [11]:
train_df['Label'].value_counts()

__label__no_error    177
__label__error        39
Name: Label, dtype: int64

In [12]:
label_ = []
message = []

for i in range(0,len(test_txt)):
  mssg = str(test_txt[test_txt.index==i])
  mssg_ = re.split(r"\s+", mssg,maxsplit=4)
  label_.append(mssg_[3])
  message.append(mssg_[4])

In [13]:
test_df = pd.DataFrame({'Message':message,'Label':label_})

In [14]:
test_df['Label'].value_counts()

__label__no_error    137
__label__error         7
Name: Label, dtype: int64

In [ ]:
############################################################# Model 1 Based on Fast Text##################################################################################

In [15]:
model = fasttext.train_supervised(input='train.txt')

In [16]:
model.test('test.txt')   

(144, 0.9513888888888888, 0.9513888888888888)

In [17]:
## Add more epochs and larger learning rate. This is just to see if the accuracy is improved or not.

In [18]:
model_1 = fasttext.train_supervised(input="train.txt", epoch=25,lr=0.95,wordNgrams=3)

In [19]:
model_1.test("test.txt")

(144, 0.9513888888888888, 0.9513888888888888)

In [ ]:
######################################################################Model 2 Using ML Algo###############################################################################

In [20]:
count_vectorizer = feature_extraction.text.CountVectorizer()

In [21]:
train_vectors = count_vectorizer.fit_transform(train_df["Message"])
test_vectors = count_vectorizer.fit_transform(test_df["Message"])

In [22]:
train_df['Label_num'] = train_df['Label'].apply(
    lambda x: 1 if x == '__label__error' else (0 if x == '__label__no_error' else None))

In [23]:
test_df['Label_num'] = test_df['Label'].apply(
    lambda x: 1 if x == '__label__error' else (0 if x == '__label__no_error' else None))

In [24]:
clf = linear_model.RidgeClassifier()
scores = model_selection.cross_val_score(clf, train_vectors, train_df["Label_num"], cv=3, scoring="f1")
scores

array([0.66666667, 0.66666667, 0.14285714])

In [ ]:
#################################################### Model 3 Using Bigram & Trigram #####################################################################################

In [25]:
train_df.head()

,Message,Label,Label_num
0,CPM,__label__no_error,0
1,EY972/11.A6DDD.HAN,__label__no_error,0
2,-PL/PMC71439EY/AUH/2223/C....,__label__no_error,0
3,-PR/PMC74892EY/AUH/2370/C....,__label__no_error,0
4,-R/PMC72439EY/AUH/985/C.EL...,__label__no_error,0


In [42]:
train_no_error = train_df[train_df['Label']=='__label__no_error']

In [44]:
train_no_error.head()

,Message,Label,Label_num
0,CPM,__label__no_error,0
1,EY972/11.A6DDD.HAN,__label__no_error,0
2,-PL/PMC71439EY/AUH/2223/C....,__label__no_error,0
3,-PR/PMC74892EY/AUH/2370/C....,__label__no_error,0
4,-R/PMC72439EY/AUH/985/C.EL...,__label__no_error,0


In [45]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

sentences = train_no_error['Message'].tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [46]:
tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))
print(tokenized_sent)

[['cpm'], ['ey972/11.a6ddd.han'], ['-pl/pmc71439ey/auh/2223/c', '....'], ['-pr/pmc74892ey/auh/2370/c', '....'], ['-r/pmc72439ey/auh/985/c.el', '...'], ['-11p/pmc01336ey/auh/1165/c', '...'], ['-12p/pmc75635ey/auh/1860/c', '...'], ['-13p/pmc70016ey/auh/2050/c', '...'], ['-21p/pmc76368ey/auh/2270/c', '...'], ['-22p/pmc72223ey/auh/2375/c', '...'], ['-23p/pmc74605ey/auh/2390/c', '...'], ['-31p/pmc74776ey/auh/2445/c', '...'], ['-33l/ake29519ey/auh/444/c', '....'], ['-33r/ake29217ey/auh/65/x'], ['-41l/ake26910ey/auh/65/x'], ['-41r/n'], ['-41p/pmc75812ey/auh/2475/c', '...'], ['-42p/pmc74004ey/auh/2405/c', '...'], ['-51/n'], ['-52/n'], ['-53/n'], ['cpm'], ['ey956/28.a6ddd.auh'], ['-ar/pmc77270ey/jnb/1540/c'], ['-bl/pmc77913ey/jnb/1235/c'], ['-br/pmc77717ey/jnb/1250/c'], ['-cl/pmc70333ey/jnb/1420/c'], ['-cr/pmc75544ey/jnb/1440/c', '....'], ['-dl/pmc76410ey/jnb/1475/c'], ['-dr/pmc76297ey/jnb/1605/c'], ['-el/pmc77378ey/jnb/1770/c'], ['-er/pmc76550ey/nbo/1820/c'], ['-fl/pmc72704ey/jnb/1920/c', '...

In [47]:
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import BigramCollocationFinder

bigrams_patterns = []

for token in tokenized_sent :
  for words in token :
    bigram_collocation = BigramCollocationFinder.from_words(words.split('/'))
    # Top 10 most occurring collocations
    bigrams_patterns.append(bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))
    print("Bigrams:", bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))

Bigrams: []
Bigrams: [('ey972', '11.a6ddd.han')]
Bigrams: [('-pl', 'pmc71439ey'), ('2223', 'c'), ('auh', '2223'), ('pmc71439ey', 'auh')]
Bigrams: []
Bigrams: [('-pr', 'pmc74892ey'), ('2370', 'c'), ('auh', '2370'), ('pmc74892ey', 'auh')]
Bigrams: []
Bigrams: [('-r', 'pmc72439ey'), ('985', 'c.el'), ('auh', '985'), ('pmc72439ey', 'auh')]
Bigrams: []
Bigrams: [('-11p', 'pmc01336ey'), ('1165', 'c'), ('auh', '1165'), ('pmc01336ey', 'auh')]
Bigrams: []
Bigrams: [('-12p', 'pmc75635ey'), ('1860', 'c'), ('auh', '1860'), ('pmc75635ey', 'auh')]
Bigrams: []
Bigrams: [('-13p', 'pmc70016ey'), ('2050', 'c'), ('auh', '2050'), ('pmc70016ey', 'auh')]
Bigrams: []
Bigrams: [('-21p', 'pmc76368ey'), ('2270', 'c'), ('auh', '2270'), ('pmc76368ey', 'auh')]
Bigrams: []
Bigrams: [('-22p', 'pmc72223ey'), ('2375', 'c'), ('auh', '2375'), ('pmc72223ey', 'auh')]
Bigrams: []
Bigrams: [('-23p', 'pmc74605ey'), ('2390', 'c'), ('auh', '2390'), ('pmc74605ey', 'auh')]
Bigrams: []
Bigrams: [('-31p', 'pmc74776ey'), ('2445', 'c

In [50]:
len(bigrams_patterns)

266

In [48]:
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures

trigrams_pattern = []

for token in tokenized_sent :
  for words in token :
    trigram_collocation = TrigramCollocationFinder.from_words(words.split('/'))
    trigrams_pattern.append(trigram_collocation.nbest(TrigramAssocMeasures.likelihood_ratio,10))
    print("Trigrams:", trigram_collocation.nbest(TrigramAssocMeasures.likelihood_ratio,10))

Trigrams: []
Trigrams: []
Trigrams: [('-pl', 'pmc71439ey', 'auh'), ('auh', '2223', 'c'), ('pmc71439ey', 'auh', '2223')]
Trigrams: []
Trigrams: [('-pr', 'pmc74892ey', 'auh'), ('auh', '2370', 'c'), ('pmc74892ey', 'auh', '2370')]
Trigrams: []
Trigrams: [('-r', 'pmc72439ey', 'auh'), ('auh', '985', 'c.el'), ('pmc72439ey', 'auh', '985')]
Trigrams: []
Trigrams: [('-11p', 'pmc01336ey', 'auh'), ('auh', '1165', 'c'), ('pmc01336ey', 'auh', '1165')]
Trigrams: []
Trigrams: [('-12p', 'pmc75635ey', 'auh'), ('auh', '1860', 'c'), ('pmc75635ey', 'auh', '1860')]
Trigrams: []
Trigrams: [('-13p', 'pmc70016ey', 'auh'), ('auh', '2050', 'c'), ('pmc70016ey', 'auh', '2050')]
Trigrams: []
Trigrams: [('-21p', 'pmc76368ey', 'auh'), ('auh', '2270', 'c'), ('pmc76368ey', 'auh', '2270')]
Trigrams: []
Trigrams: [('-22p', 'pmc72223ey', 'auh'), ('auh', '2375', 'c'), ('pmc72223ey', 'auh', '2375')]
Trigrams: []
Trigrams: [('-23p', 'pmc74605ey', 'auh'), ('auh', '2390', 'c'), ('pmc74605ey', 'auh', '2390')]
Trigrams: []
Trigr

In [49]:
len(trigrams_pattern)

266

In [51]:
test_error = test_df[test_df['Label']=='__label__error']

In [53]:
test_no_error = test_df[test_df['Label']== '__label__no_error']

In [54]:
test_no_error

,Message,Label,Label_num
2,SI,__label__no_error,0
3,BULK/AUH/1764/C,__label__no_error,0
4,END,__label__no_error,0
5,CPM,__label__no_error,0
6,EY015/27.A6BMB.AUH,__label__no_error,0
...,...,...,...
136,14/Y2 MIX,__label__no_error,0
137,15R/P2 MIX,__label__no_error,0
138,12L/G1 MIX,__label__no_error,0
139,12R/S MIX,__label__no_error,0


In [52]:
test_error

,Message,Label,Label_num
0,-41P/PMC74049EY/AUH/3576/C/LD,__label__error,1
1,-42P/PMC74557EY/AUH/3652/C/LD,__label__error,1
122,-44L/AKE25133EY/AUH/1210/C.HE...,__label__error,1
123,HEA.PEP,__label__error,1
140,BULK,__label__error,1
141,BJ-09//G2 -09//G1-06//R2-05,__label__error,1
143,"""""",__label__error,1


In [ ]:
## Splitting in bigram and trigram for the test data.

In [55]:
sentences = test_no_error['Message'].tolist()

tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))
print(tokenized_sent)

[['si'], ['bulk/auh/1764/c'], ['end'], ['cpm'], ['ey015/27.a6bmb.auh'], ['-21p/n'], ['-22p/pmc71680ey/auh/1820/x', '...'], ['-23p/pmc76052ey/auh/1869/m'], ['-31p/pmc75338ey/auh/2340/x', '...'], ['-32p/n'], ['-41l/ake26177ey/auh/113/c'], ['-41r/n'], ['-42l/ake23927ey/auh/609/m'], ['-42r/ake29769ey/auh/953/m'], ['-43/alf41072ey/auh/1534/m'], ['-44l/600/e.eic'], ['-44r/ake23963ey/auh/778/m'], ['-51/n'], ['-52/n'], ['-53/n'], ['si', '44l', 'fsk'], ['cpm'], ['ey917/08.a6ddb.auh'], ['-al/paj10367ey/ams/2207/c', '....'], ['-ar/pmc70521ey/ams/1925/c'], ['-bl/hml00610ej/ams/2850/c', '....'], ['-br/pmc77052ey/ams/1505/c', '....'], ['-cl/hml00662ej/ams/2710/c', '....'], ['-cr/pmc73662ey/ams/1780/c', '....'], ['-dl/hml00683ej/ams/2710/c', '....'], ['-dr/pmc79883ey/ams/2130/c'], ['-el/hml00914ej/ams/2710/c', '....'], ['-er/pmc72276ey/ams/2340/c'], ['-fl/hml00920ej/ams/2710/c', '....'], ['-11p/pmc71010ey/man/1274/c'], ['-12l/ake/man/551/bj1-12r/n'], ['-13l/ake61670ey/man/692/by', '...'], ['-14l/ake7

In [56]:
bigrams_patterns_test = []

for token in tokenized_sent :
  for words in token :
    bigram_collocation = BigramCollocationFinder.from_words(words.split('/'))
    # Top 10 most occurring collocations
    bigrams_patterns_test.append(bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))
    print("Bigrams:", bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))

Bigrams: []
Bigrams: [('1764', 'c'), ('auh', '1764'), ('bulk', 'auh')]
Bigrams: []
Bigrams: []
Bigrams: [('ey015', '27.a6bmb.auh')]
Bigrams: [('-21p', 'n')]
Bigrams: [('-22p', 'pmc71680ey'), ('1820', 'x'), ('auh', '1820'), ('pmc71680ey', 'auh')]
Bigrams: []
Bigrams: [('-23p', 'pmc76052ey'), ('1869', 'm'), ('auh', '1869'), ('pmc76052ey', 'auh')]
Bigrams: [('-31p', 'pmc75338ey'), ('2340', 'x'), ('auh', '2340'), ('pmc75338ey', 'auh')]
Bigrams: []
Bigrams: [('-32p', 'n')]
Bigrams: [('-41l', 'ake26177ey'), ('113', 'c'), ('ake26177ey', 'auh'), ('auh', '113')]
Bigrams: [('-41r', 'n')]
Bigrams: [('-42l', 'ake23927ey'), ('609', 'm'), ('ake23927ey', 'auh'), ('auh', '609')]
Bigrams: [('-42r', 'ake29769ey'), ('953', 'm'), ('ake29769ey', 'auh'), ('auh', '953')]
Bigrams: [('-43', 'alf41072ey'), ('1534', 'm'), ('alf41072ey', 'auh'), ('auh', '1534')]
Bigrams: [('-44l', '600'), ('600', 'e.eic')]
Bigrams: [('-44r', 'ake23963ey'), ('778', 'm'), ('ake23963ey', 'auh'), ('auh', '778')]
Bigrams: [('-51', 'n'

In [57]:
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures

trigrams_pattern_test = []

for token in tokenized_sent :
  for words in token :
    trigram_collocation = TrigramCollocationFinder.from_words(words.split('/'))
    trigrams_pattern_test.append(trigram_collocation.nbest(TrigramAssocMeasures.likelihood_ratio,10))
    print("Trigrams:", trigram_collocation.nbest(TrigramAssocMeasures.likelihood_ratio,10))

Trigrams: []
Trigrams: [('auh', '1764', 'c'), ('bulk', 'auh', '1764')]
Trigrams: []
Trigrams: []
Trigrams: []
Trigrams: []
Trigrams: [('-22p', 'pmc71680ey', 'auh'), ('auh', '1820', 'x'), ('pmc71680ey', 'auh', '1820')]
Trigrams: []
Trigrams: [('-23p', 'pmc76052ey', 'auh'), ('auh', '1869', 'm'), ('pmc76052ey', 'auh', '1869')]
Trigrams: [('-31p', 'pmc75338ey', 'auh'), ('auh', '2340', 'x'), ('pmc75338ey', 'auh', '2340')]
Trigrams: []
Trigrams: []
Trigrams: [('-41l', 'ake26177ey', 'auh'), ('ake26177ey', 'auh', '113'), ('auh', '113', 'c')]
Trigrams: []
Trigrams: [('-42l', 'ake23927ey', 'auh'), ('ake23927ey', 'auh', '609'), ('auh', '609', 'm')]
Trigrams: [('-42r', 'ake29769ey', 'auh'), ('ake29769ey', 'auh', '953'), ('auh', '953', 'm')]
Trigrams: [('-43', 'alf41072ey', 'auh'), ('alf41072ey', 'auh', '1534'), ('auh', '1534', 'm')]
Trigrams: [('-44l', '600', 'e.eic')]
Trigrams: [('-44r', 'ake23963ey', 'auh'), ('ake23963ey', 'auh', '778'), ('auh', '778', 'm')]
Trigrams: []
Trigrams: []
Trigrams: [

In [81]:
indi_patt = []

for pat in bigrams_patterns :
  for indi_pat in pat : 
    for i in indi_pat :
      indi_patt.append(i)

In [82]:
indi_patt_test = []

for pat in bigrams_patterns_test :
  for indi_pat in pat : 
    for i in indi_pat :
      indi_patt_test.append(i)

In [69]:
indi_patt[0:5]

['ey972', '11.a6ddd.han', '-pl', 'pmc71439ey', '2223']

In [70]:
indi_patt_test[0:5]

['1764', 'c', 'auh', '1764', 'bulk']

In [ ]:
#!pip install --target=$nb_path fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz

In [73]:
fuzz.ratio(indi_patt,indi_patt_test)

3

In [74]:
fuzz.partial_ratio(indi_patt,indi_patt_test)

32

In [76]:
fuzz.token_sort_ratio(indi_patt,indi_patt_test)

14

In [83]:
## Random Shuffling 
import random

indi_patt = random.shuffle(indi_patt)
indi_patt_test = random.shuffle(indi_patt_test)

In [84]:
fuzz.ratio(indi_patt,indi_patt_test)

0

In [85]:
fuzz.partial_ratio(indi_patt,indi_patt_test)

0

In [86]:
fuzz.token_sort_ratio(indi_patt,indi_patt_test)

0

In [ ]:
#############Trigram Fuzzy Score Methodology###############################

In [88]:
indi_patt = []

for pat in trigrams_pattern :
  for indi_pat in pat : 
    for i in indi_pat :
      indi_patt.append(i)

In [89]:
indi_patt_test = []

for pat in trigrams_pattern_test :
  for indi_pat in pat : 
    for i in indi_pat :
      indi_patt_test.append(i)

In [90]:
fuzz.ratio(indi_patt,indi_patt_test)

2

In [91]:
fuzz.partial_ratio(indi_patt,indi_patt_test)

28

In [92]:
fuzz.token_sort_ratio(indi_patt,indi_patt_test)

19

In [93]:
## Random Shuffling 
import random

indi_patt = random.shuffle(indi_patt)
indi_patt_test = random.shuffle(indi_patt_test)

In [94]:
fuzz.ratio(indi_patt,indi_patt_test)

0

In [95]:
fuzz.partial_ratio(indi_patt,indi_patt_test)

0

In [96]:
fuzz.token_sort_ratio(indi_patt,indi_patt_test)

0

In [ ]:
#################################################################################### The End ##################################################################